In [90]:
import numpy as np 
import pandas as pd 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from csv import reader
from sklearn.preprocessing import MinMaxScaler
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
torch.__version__

'1.6.0'

In [78]:
# def load_data(filepath,n_train_data=1096):
#     dataset = []
#     with open(filepath,'r') as f:
#         csv_reader =reader(f,delimiter = ',')
#         for row in csv_reader:
#             row[1:] = list(map(float,row[1:]))
#             #row[0] = int(row[0])
#             dataset.append(row[1:])
#         dataset = np.array(dataset)
#         #归一化
#         # mms = MinMaxScaler()
#         # for i in range(dataset.shape[0]-1):
#         #     dataset[:,i] = mms.fit_transform(dataset[:,i].reshape(-1,1)).flatten()
#         #print(dataset[0])
        
#         #split_data
#         train_data = dataset[0:n_train_data]
#         val_data = dataset[n_train_data:]  

#         return train_data,val_data

# train_data,test_data = load_data('./ele_pre.csv')   
# print(train_data[0])
# print(test_data[0])

In [93]:
#对14年12月和15年 前10天的电力负载进行预测
DAYS_FOR_TRAIN = 365
class LSTM_Regression(nn.Module):
    """
        使用LSTM进行回归
        
        参数：
        - input_size: feature size
        - hidden_size: number of hidden units
        - output_size: number of output
        - num_layers: layers of LSTM to stack
    """
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, _x):
        x, _ = self.lstm(_x)  # _x is input, size (seq_len, batch, input_size)
        s, b, h = x.shape  # x is output, size (seq_len, batch, hidden_size)
        x = x.view(s*b, h)
        x = self.fc(x)
        x = x.view(s, b, -1)  # 把形状改回来
        return x

class BPNet(nn.Module):
    def __init__(self):
        super(BPNet,self).__init__()
        self.lr1 = nn.Linear(5,1000)        
        self.lr2 = nn.Linear(1000,2000)        
        self.lr3 = nn.Linear(2000,512)        
        self.lr4 = nn.Linear(512,100)        
        self.lr5 = nn.Linear(100,10)        
        self.lr6 = nn.Linear(10,1)        
    def forward(self,x):
        out = self.lr1(x)
        out = F.relu(out)
        out = self.lr2(out)
        out = F.relu(out)
        out = self.lr3(out)
        out = F.relu(out)
        out = self.lr4(out)
        out = F.relu(out)
        out = self.lr5(out)
        out = F.relu(out)
        out = self.lr6(out)
        return out

model = BPNet()
#model = LSTM_Regression(DAYS_FOR_TRAIN, 8, output_size=1, num_layers=2)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


In [80]:
# def create_dataset(data, days_for_train=1):
#     """
#         根据给定的序列data，生成数据集
        
#         数据集分为输入和输出，每一个输入的长度为days_for_train，每一个输出的长度为1。
#         也就是说用days_for_train天的数据，对应下一天的数据。
#         若给定序列的长度为d，将输出长度为(d-days_for_train+1)个输入/输出对
#     """
#     dataset_x, dataset_y= [], []
#     for i in range(len(data)-days_for_train):
#         _x = data[i:(i+days_for_train)]
#         dataset_x.append(_x)
#         dataset_y.append(data[i+days_for_train])
#     return (np.array(dataset_x), np.array(dataset_y))
# print(ele[0],type(ele[0]))
# dataset_x, dataset_y = create_dataset(ele, DAYS_FOR_TRAIN)
# #dataset_y 为下一年的数据
# # print(dataset_x[0][0])
# # print(dataset_y[0])
# train_size = int(len(dataset_x) * 0.7)
# train_x = dataset_x[:train_size]
# train_y = dataset_y[:train_size]
# # 将数据改变形状，RNN 读入的数据维度是 (seq_size, batch_size, feature_size)
# train_x = train_x.reshape(-1, 1, DAYS_FOR_TRAIN)
# train_y = train_y.reshape(-1, 1, 1)
# # 转为pytorch的tensor对象
# train_x = torch.from_numpy(train_x)
# train_y = torch.from_numpy(train_y)
# train_data = torch.from_numpy(train_data)

pre_data = np.genfromtxt('ele_pre.txt',delimiter='\t',dtype=float)
# print(pre_data[0])
train_x = pre_data[:,1:6]
train_x = torch.from_numpy(train_x)
train_x = torch.tensor(train_x,dtype=torch.float32)
train_y = pre_data[:,6:7]
# train_y[0]
train_y = torch.from_numpy(train_y)
train_y = torch.tensor(train_y,dtype=torch.float32)



<ipython-input-80-edb90817bfc9>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_x = torch.tensor(train_x,dtype=torch.float32)
<ipython-input-80-edb90817bfc9>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_y = torch.tensor(train_y,dtype=torch.float32)


In [94]:
for i in range(1000):  
    optimizer.zero_grad()
    #print(train_x.shape)                 
    out = model(train_x)
    loss = loss_function(out,train_y)
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
        print('Epoch: {}, Loss:{:.5f}'.format(i+1, loss.item()))

Epoch: 100, Loss:nan
Epoch: 200, Loss:nan
Epoch: 300, Loss:nan


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
model = model.eval() # 转换成测试模式
# 注意这里用的是全集 模型的输出长度会比原数据少DAYS_FOR_TRAIN 填充使长度相等再作图
dataset_x = dataset_x.reshape(-1, 1, DAYS_FOR_TRAIN)  # (seq_size, batch_size, feature_size)
dataset_x = torch.from_numpy(dataset_x)
pred_test = model(dataset_x) # 全量训练集的模型输出 (seq_size, batch_size, output_size)
pred_test = pred_test.view(-1).data.numpy()
pred_test = np.concatenate((np.zeros(DAYS_FOR_TRAIN), pred_test))  # 填充0 使长度相同
assert len(pred_test) == len(ele)
plt.plot(pred_test, 'r', label='prediction')
plt.plot(ele, 'b', label='real')
plt.plot((train_size, train_size), (0, 1), 'g--')
plt.legend(loc='best')
plt.show()

In [ ]:
# import csv

# csvFile = open("./ele_pre.csv",'w',newline='',encoding='utf-8')

# writer = csv.writer(csvFile)

# csvRow = []
# f = open("ele_pre.txt",'r',encoding='GB2312')

# for line in f:
#     csvRow = line.split()

#     writer.writerow(csvRow)
# f.close()
# csvFile.close()